# Make parameter submission scripts for HPC 

## $\xi$ sweep

Author: Juvid Aryaman

In [17]:
import numpy as np
import pandas as pd
import mitonetworks.det as mtd
import mitonetworks.stoch as mts

In [18]:
import warnings
warnings.simplefilter("ignore",RuntimeWarning)

In [19]:
nominal_params = {
    'gamma':0.03785142857142858,
	'beta':33.12,
	'kappa':11.662903457629223,
	'b':1.2416523075924095e-05,
	'mu':0.023,
	'xi':0.0,
	'delta':1.0
}

In [20]:
c_file_param_ordering_convention = ['xi','beta','gamma','kappa','b','mu','delta']

In [21]:
hpc_workdir = '$WORK/networks/sing_birth_fus/xi_sw'

In [22]:
c_filename = 'closed_loop_deg_single'

In [23]:
hpc_jobname = 'xi_sw'

In [24]:
xs = mts.SubmitHPCSoluableFeedbackControl(ss_definition=mtd.E_linear_feedback_control_ss,
                                 nominal_params=nominal_params,hpc_workdir=hpc_workdir,
                                c_filename=c_filename, c_file_param_ordering_convention=c_file_param_ordering_convention,
                                hpc_jobname=hpc_jobname)

In [26]:
ics, h_opt, err = xs.find_ss_h_target(nominal_params)
print((ics[0]+[2])/float(np.sum(ics)))

[0.32832833]


Find optimal $\kappa,\xi$ pairs for $h=0.3$ and $n=1000$

In [32]:
n_target = 1000.0
kappa_space = np.linspace(-1000,1000,2001)
xi_sp = [0.2,0.4,0.6,0.8]

opt_params = []

params = nominal_params.copy()

for xi_val in xi_sp:
    params['xi'] = xi_val
    
    n_sp = []
    n_ind_sp = []
    fs_sp = []
    ics_sp = []
    
    # Find corresponding kappa to keep copy number constant
    for j, kappa_val in enumerate(kappa_space): 
        params['kappa'] = kappa_val
        ics, h_opt, err = xs.find_ss_h_target(params)
        
        if err == 1:
            n_sp.append(np.sum(ics))
            n_ind_sp.append(j)
            fs_sp.append((ics[0]+[2])/float(np.sum(ics)))
            ics_sp.append(ics)
        else:
            continue
    n_sp = np.array(n_sp)
    n_ind_opt =  np.argmin(np.abs(n_sp - n_target))
    
    print(n_sp[n_ind_opt],fs_sp[n_ind_opt]) # fs at this optimal parametrization. This should be invariant
    
    if abs(n_sp[n_ind_opt] - n_target) > 1.5:
        raise Exception("Could not find a kappa for this mu where n is close to n_target")
    
    opt_params.append([params['xi'],kappa_space[n_ind_sp[n_ind_opt]],ics_sp[n_ind_opt]])
    

(999, array([0.32832833]))
(999, array([0.32832833]))
(1000, array([0.328]))
(999, array([0.32832833]))


In [33]:
opt_params

[[0.2, 209.0, array([326, 373, 140, 160])],
 [0.4, 406.0, array([326, 373, 140, 160])],
 [0.6, 605.0, array([326, 374, 140, 160])],
 [0.8, 801.0, array([326, 373, 140, 160])]]

Make submission

In [36]:
master_script = open(xs.out_dir+"/master.sh","w")
# Add the .c compilation line to the master script
master_script.write('gcc -o3 {0}.c -lm -o {0}.ce\n'.format(xs.c_filename))

# Output dataframe with parametriations
param_df =  []

for i, p in enumerate(opt_params):
    params['xi'] = p[0]
    params['kappa'] = p[1]
    param_dict = params.copy()
    
    ics, h_opt, err = xs.find_ss_h_target(params)
    
    for k, param in enumerate(xs.param_convention): param_dict[param+'_init'] = ics[k]
    param_df.append(pd.DataFrame(param_dict, index=[0]))
    
    ics, h_opt, err = xs.find_ss_h_target(params)
    if err == 1:
        xs.make_submission_script(i,ics,params)
        # Add the submission to a master script
        master_script.write('echo "{}"\n'.format(i))
        master_script.write('mkdir -p {0}/p{1}\n'.format(xs.hpc_workdir,i))
        master_script.write("qsub submission_{}.pbs\n".format(i))
        master_script.write('sleep 0.5\n')
    else:
        raise Exception('SS not found!')
master_script.close()
param_df = pd.concat(param_df, ignore_index=True)
param_df.to_csv(xs.out_dir+'/param_sweep_vals.csv',index=False)    

In [37]:
param_df

,b,beta,delta,gamma,kappa,mf_init,ms_init,mu,wf_init,ws_init,xi
0,0.000012,33.12,1.0,0.037851,209.0,160,140,0.023,373,326,0.2
1,0.000012,33.12,1.0,0.037851,406.0,160,140,0.023,373,326,0.4
2,0.000012,33.12,1.0,0.037851,605.0,160,140,0.023,374,326,0.6
3,0.000012,33.12,1.0,0.037851,801.0,160,140,0.023,373,326,0.8
